In [1]:
import pandas as pd

In [2]:
# Считываем данные из файла
data=pd.read_csv("./Data/casc-resto.csv",sep=';')

C:\Users\tagil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Смотрим на данные по нужной дате
print(data[data['RKDate'] == '2017-07-01'])

        CustomerID  Restaurant      RKDate                    RegionName  \
4665       2810585         436  2017-07-01  Москва и Московская область    
5606       2848543         727  2017-07-01  Москва и Московская область    
23504      2795631          49  2017-07-01  Москва и Московская область    
23971      2831289          93  2017-07-01  Москва и Московская область    
24618      2776418         139  2017-07-01  Москва и Московская область    
...            ...         ...         ...                           ...   
881148     2793183          43  2017-07-01  Москва и Московская область    
881323     2875013         922  2017-07-01  Москва и Московская область    
881648     2880345         449  2017-07-01  Москва и Московская область    
881867     2908412         469  2017-07-01  Москва и Московская область    
882131     2777471         464  2017-07-01  Москва и Московская область    

                  BrandsNames DishCategoryName  Quantity SummBasic  \
4665             

In [6]:
# Группируем данные по идентификатору клиента и смотрим на данные
grouped_data=data.groupby('CustomerID')
print(grouped_data.head())

        CustomerID  Restaurant      RKDate                    RegionName  \
0          2898197         391  2017-07-16  Москва и Московская область    
1          2903215          43  2015-04-07  Москва и Московская область    
2          2748887          43  2015-05-22  Москва и Московская область    
3          2862077          46  2015-03-05  Москва и Московская область    
4          2862077          46  2015-03-11  Москва и Московская область    
...            ...         ...         ...                           ...   
848938     2822873         920  2015-05-23  Москва и Московская область    
866131     2882296         722  2017-08-07  Москва и Московская область    
874643     2771310         128  2016-02-02  Москва и Московская область    
878930     2777572         440  2015-08-22  Москва и Московская область    
880274     2787762          49  2016-09-01  Москва и Московская область    

                  BrandsNames DishCategoryName  Quantity SummBasic  \
0                

In [9]:
for  name, group in grouped_data:
    print(len(group[group['RKDate'] < '2017-07-01' and group['RKDate'] > '2017-12-31'   ] ))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# функция для получения флага "покупал или не покупал" 
def boughtOrNot(dates):
    for date in dates:
        if date<='2017-12-31' and date>='2017-07-01':
            return 1
        return 0
a={}
a['bought']=boughtOrNot(group['RKDate'])
# функция для получения показателя Recency
def getRecency(dates):
    for date in dates:
        #if date<='2017-07-01'
# функция для получения показателя Frequency
# функция для получения показателя Monetary Value
# функция для вычисления собственного показателя
#